In [1]:
import pandas as pd
import numpy as np
import time_series_transform.base as base
from time_series_transform.time_series_transformer import *

In [2]:
df = pd.DataFrame([{'t1':1,'t2':2,'t3':3,'t4':4,'cate1':30},{'t1':1,'t2':2,'t3':3,'t4':4,'cate1':30}])

In [3]:
data = df.to_dict('records')[0]

In [4]:
config = {}
config['series'] = {
    'colNames':['t1','t2','t3','t4'],
    'tensorType':'sequence',
    'param':{
        'windowSize':2,
        'outType':np.float32
    }
}
config['label'] = {
    'colNames':['t1','t2','t3','t4'],
    'tensorType':'label',
    'param':{
        'windowSize':2,
        'outType':np.float32
    }
}
config['series_2'] = {
    'colNames':['t1','t2','t3','t4'],
    'tensorType':'sequence',
    'param':{
        'windowSize':2,
        'outType':np.float32
    },
    'sequence_stack':"series"
}

In [5]:
pst = Pandas_Time_Series_Dataset(df,config)

In [6]:
pst._make_time_series_dataset(data)

[None, <time_series_transform.base.Time_Series_Tensor at 0x154eed71f60>]